In [1]:
import numpy as np
import time
import csv
import random

def candidatos(amostra, restricao):

    if restricao == True:

        possiveis = np.zeros(1)
        for i in range(len(amostra)):
            if i == 0:
                if amostra[i+1] == 0 and amostra[i] == 0:
                    possiveis = np.append(possiveis,i)
            else:
                if i == len(amostra)-1:
                    if amostra[i-1] == 0 and amostra[i] == 0:
                        possiveis = np.append(possiveis,i)
                else:
                    if amostra[i+1] == 0 and amostra[i-1] == 0 and amostra[i] == 0:
                        possiveis = np.append(possiveis,i)
    
    else:

        possiveis = np.zeros(1)
        for i in range(len(amostra)):
            if amostra[i] == 0:
                possiveis = np.append(possiveis,i)
    
    if len(possiveis) == 1:
        return np.array([-1])
        
    possiveis = np.delete(possiveis,0)

    return possiveis

def candidatosrestritos(amostra, restricao):
    
    possiveis = candidatos(amostra, restricao)

    if len(possiveis) == 1:
        if possiveis[0] == -1:
            return np.array([-1])

    restritos = np.zeros(1)
        
    melhor_indice = -1
    melhor_resultado = 0
    desempenho_anterior = amostra @ custos @ amostra
        
    for j in range(len(possiveis)):
        teste = np.copy(amostra)
        teste[int(possiveis[j])] = 1
        desempenho = teste @ custos @ teste
        if desempenho - desempenho_anterior > melhor_resultado:
            melhor_resultado = desempenho - desempenho_anterior
            melhor_indice = possiveis[j]

    if melhor_indice >= 0:
            
        limite_inferior = melhor_resultado*alfa
        
        for j in range(len(possiveis)):
            teste = np.copy(amostra)
            teste[int(possiveis[j])] = 1
            desempenho = teste @ custos @ teste
            if desempenho - desempenho_anterior >= limite_inferior:
                restritos = np.append(restritos,possiveis[j])

    if len(restritos) == 1:
            return np.array([-1])
    restritos = np.delete(restritos,0)
    return restritos

def nova_escolha(amostra, restricao):
    for i in range(len(amostra)-1):
        restritos = candidatosrestritos(amostra, restricao)
        if len(restritos) == 1:
            if restritos[0] == -1:
                break
        escolha = random.randint(0,len(restritos)-1)
        amostra[int(restritos[escolha])] = 1
    return amostra

def formulacao_inicial(custos, restricao, repeticoes):

    melhor_desempenho_construtivo = -1
    melhor_amostra_construtiva = np.zeros(len(custos))
    
    for rep in range(repeticoes):

        amostra = np.zeros(len(custos))

        # Primeiro chute
        k = random.randint(0,len(amostra)-1)
        amostra[k] = 1

        # Proximas escolhas
        amostra_original = nova_escolha(amostra, restricao)
        desempenho_original = amostra_original @ custos @ amostra_original

        if desempenho_original > melhor_desempenho_construtivo:
            melhor_desempenho_construtivo = desempenho_original
            melhor_amostra_construtiva = amostra_original
        
    return melhor_amostra_construtiva, melhor_desempenho_construtivo

def varredura_local(amostra_original, desempenho_original, restricao, repeticoes, aleatoriedade, tipo):
    
    melhor_desempenho_global = 0
    if tipo == 'first':
        for rep in range(repeticoes):
            amostra_copia = np.copy(amostra_original)
            for i in range(len(amostra_copia)):
                if amostra_copia[i] == 1:
                    if random.randint(1,aleatoriedade) == 1:
                        amostra_copia[i] = 0
            amostra_copia = nova_escolha(amostra_copia, restricao)
            desempenho_copia = amostra_copia @ custos @ amostra_copia

            if desempenho_copia > desempenho_original:
                amostra_original = amostra_copia
                desempenho_original = desempenho_copia
                rep = 0
    
            if desempenho_original > melhor_desempenho_global:
                melhor_desempenho_global = desempenho_original
                melhor_amostra_global = amostra_original
    
    if tipo == 'best':
        melhor_desempenho_copia = desempenho_original
        for rep in range(repeticoes):
            amostra_copia = np.copy(amostra_original)
            for i in range(len(amostra_copia)):
                if amostra_copia[i] == 1:
                    if random.randint(1,aleatoriedade) == 1:
                        amostra_copia[i] = 0
            amostra_copia = nova_escolha(amostra_copia, restricao)
            desempenho_copia = amostra_copia @ custos @ amostra_copia

            if desempenho_copia > melhor_desempenho_copia:
                melhor_desempenho_copia = desempenho_copia
                melhor_amostra_copia = amostra_copia

            if rep == repeticoes - 1:
                if melhor_desempenho_copia != desempenho_original:
                    rep = 0
                    amostra_original = melhor_amostra_copia
                    desempenho_original = melhor_desempenho_copia


        if desempenho_original > melhor_desempenho_global:
            melhor_desempenho_global = desempenho_original
            melhor_amostra_global = amostra_original

    return melhor_amostra_global, melhor_desempenho_global

def texto(restricao, metodo, tipo_de_varredura, custos, tempo, alfa, aleatoriedade, melhor_desempenho_global, melhor_amostra_global, melhor_desempenho_construtivo, melhor_amostra_construtiva):
    if restricao == True:
        print('Analise restrita do qbf', len(custos))
    else:
        print('Analise irrestrita do qbf', len(custos))

    print('\nTempo de execução: ', time.time()-tempo, 'segundos')
    print('\nAlfa: ',alfa)
    print('Probabilidade: ',1/aleatoriedade)
    print('Metodo de contrução: ',metodo)
    print('Tipo de varredura: ',tipo_de_varredura)
    print('\nMelhor desempenho global: ', melhor_desempenho_global)
    print('Melhor amostra global: \n', melhor_amostra_global)
    print('\nMelhor desempenho construtivo: ', melhor_desempenho_construtivo)
    print('Melhor amostra construtiva: \n', melhor_amostra_construtiva)

def solucao(metodo, tipo_de_varredura, repet1, repet2):
    
    tempo = time.time()

    melhor_desempenho_global = -1
    melhor_desempenho_construtivo = -1

    if metodo == 'Random plus greedy':
        for i in range(repet1):
            
            amostra_original, desempenho_original = formulacao_inicial(custos, restricao, repeticoes=1)
            amostra_varredura, desempenho_varredura = varredura_local(amostra_original, desempenho_original, restricao, repeticoes=repet2, aleatoriedade=aleatoriedade,tipo=tipo_de_varredura)
            
            if desempenho_original > melhor_desempenho_construtivo:
                melhor_desempenho_construtivo = desempenho_original
                melhor_amostra_construtiva = amostra_original

            if desempenho_varredura > melhor_desempenho_global:
                melhor_desempenho_global = desempenho_varredura
                melhor_amostra_global = amostra_varredura

    if metodo == 'Sampled greedy construction':
        for i in range(1):
            
            amostra_original, desempenho_original = formulacao_inicial(custos, restricao, repeticoes=repet1)
            amostra_varredura, desempenho_varredura = varredura_local(amostra_original, desempenho_original, restricao, repeticoes=repet2, aleatoriedade=aleatoriedade,tipo=tipo_de_varredura)
            
            if desempenho_original > melhor_desempenho_construtivo:
                melhor_desempenho_construtivo = desempenho_original
                melhor_amostra_construtiva = amostra_original

            if desempenho_varredura > melhor_desempenho_global:
                melhor_desempenho_global = desempenho_varredura
                melhor_amostra_global = amostra_varredura
    
    texto(restricao, metodo, tipo_de_varredura, custos, tempo, alfa, aleatoriedade, melhor_desempenho_global, melhor_amostra_global, melhor_desempenho_construtivo, melhor_amostra_construtiva)

# Abrindo o arquivo é adicionando 0 na parte inferior da matriz
contador = 0
data = []
caminho_arquivo = r"C:\Users\pedro\Desktop\instances\qbf020"
with open(caminho_arquivo, "r") as fileraw:
    file = csv.reader(fileraw, delimiter = " ")
    for linha in file:
        if contador !=0:
            if contador == 1:
                custos = np.zeros((int(len(linha)),int(len(linha))))
                for i in range(len(linha)):
                    custos[contador-1,i] = linha[i]
                    #custos[i, contador-1] = linha[i] #Se for espelhado
            else:
                for i in range(len(linha)):
                    custos[contador-1,i+contador-1] = linha[i]
                    #custos[i+contador-1,contador-1] = linha[i] #Se for espelhado
        contador += 1

restricao = False # Restrição de 1 adjacentes a outro 1
alfa = 0.6 # limite inferior = melhor*alfa
aleatoriedade = 5 # probabilidade = 1/aleatoriedade
tipo_de_varredura = 'first' # best ou first
metodo = 'Random plus greedy' # Random plus greedy ou Sampled greedy construction

solucao(metodo, tipo_de_varredura, repet1 = 100, repet2 = 100) # repet1 é o parametro de repetição da etapa de construção e o repet2 é o parametro da varredura

Analise irrestrita do qbf 20

Tempo de execução:  4.172738552093506 segundos

Alfa:  0.6
Probabilidade:  0.2
Metodo de contrução:  Random plus greedy
Tipo de varredura:  first

Melhor desempenho global:  151.0
Melhor amostra global: 
 [0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0.]

Melhor desempenho construtivo:  151.0
Melhor amostra construtiva: 
 [0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0.]
